In [ ]:
using Piccolo

using LinearAlgebra
using Random

using CairoMakie
using ForwardDiff

# Example II.1
-----
**How the KKT system works**

Last time, we learned about gradient descent and Newton's method. We also learned that, without guarantees on convexity, we needed techniques like regularization and line search to actually make sure our optimization routines worked.

In this section, we are going to learn about *constrained optimization*. Let's keep things simple, and try to avoid all of the regularization and line search stuff. In the next cells, we'll define a 2D bowl as our cost function, and we'll draw some nice level curves to visualize it--it's a convex cost, so we know it will have a minimum at the bottom of the bowl. To make it interesting, we will add a single constraint, which we draw as a level curve.

In [ ]:
Q = Diagonal([0.5; 1])

## Objective
function J(x)
    return 1 / 2 * (x - [1; 0])' * Q * (x - [1; 0])
end

function ∇J(x)
    return Q * (x - [1; 0])
end

function ∇²J(x)
    return Q
end

## Linear constraint -- you can try this, also.
# A = [1.0 -1.0]
# b = -1.0
# function f(x)
#     return A * x - b
# end

# function ∂f(x)
#     return A
# end

## Nonlinear constraint
function f(x)
    return x[1]^2 + 2*x[1] - x[2]
end

function ∂f(x)
    return [2*x[1]+2 -1]
end

In [ ]:
function draw_contour(ax; samples=40, levels=25)
    cols = kron(ones(samples), range(-4, 4, samples)')
    rows = kron(ones(samples)', range(-4, 4, samples))
    vals = zeros(samples,samples)
    for j = 1:samples
        for k = 1:samples
            vals[j, k] = J([cols[j, k]; rows[j, k]])
        end
    end
    contour!(ax, vec(cols), vec(rows), vec(vals), levels=levels)

    ## Linear x - y + 1 = 0 -- uncomment this if you want to try linear constraint
    # constraint = range(-4, 3, samples)
    # lines!(ax, constraint, constraint .+ 1, color=:black, linewidth=2)

    ## Nonlinear x^2 + 2x - y = 0
    constraint = range(-3.2, 1.2, samples)
    lines!(ax, constraint, constraint.^2 .+ 2*constraint, color=:black, linewidth=2)
end

In [ ]:
function newton_step(xᵢ, λᵢ)
    ∂²L_∂x² = ∇²J(xᵢ) + ForwardDiff.jacobian(x -> ∂f(x)'λᵢ, xᵢ)
    ∂f_∂x = ∂f(xᵢ)

    ## KKT system
    H = [∂²L_∂x² ∂f_∂x'; ∂f_∂x 0]
    g = [∇J(xᵢ) + ∂f_∂x'λᵢ; f(xᵢ)]
    
    Δz = -H\g
    Δx = Δz[1:2]
    Δλ = Δz[3]
    return xᵢ .+ Δx, λᵢ .+ Δλ
end

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], aspect=1)

## Initial guess
# xᵢ = Float64[-0.75; -1.75]
xᵢ = Float64[-3; 2]
λᵢ = Float64[0.0]

## Draw the initial contours and the initial guess
draw_contour(ax)
plot!(ax, [xᵢ[1]], [xᵢ[2]], color=:red, marker=:circle, markersize=15)
fig

In [ ]:
xᵢ₊₁, λᵢ₊₁ = newton_step(xᵢ, λᵢ)
plot!(ax, [xᵢ₊₁[1]], [xᵢ₊₁[2]], color=:red, marker=:x, markersize=15)
xᵢ .= xᵢ₊₁
λᵢ .= λᵢ₊₁

fig

In [ ]:
## Inspect the Hessian
H = ∇²J(xᵢ) + ForwardDiff.jacobian(x -> ∂f(x)'λᵢ, xᵢ)

We need regularization... even though we picked a convex cost! The constraint in our system messed things up. Let's add regularization, but we will do so a bit differently.

Recall from Part I, where we learned about LBFGS approximation to Newton's method, and how the LBFGS seemed to have robustness properites that Newton's method lacked. We will make a similar approximation to our KKT system, called the *Gauss-Newton approximation*. See the exercises below for a more detailed explanation.

The thought process is as follows: After inspecting `∂²L_∂x² = ∇²J(xᵢ) + ForwardDiff.jacobian(x -> ∂f(x)'λᵢ, xᵢ)`, we note that $\nabla^2 J$ is convex by construction. It is just the `ForwardDiff.jacobian(x -> ∂f(x)'λᵢ, xᵢ)` that causes trouble with the Hessian. At this time, we also notice that latter term is also expensive to compute. Because it causes trouble and is costly to compute, we decide to drop this term. This is the Gauss-Newton approximation. Its steps compute faster, but converge slower than Newton--luckily, the savings in compute speed often overtake any reduction in convergence rate!

In [ ]:
function gauss_newton_step(xᵢ, λᵢ)
    ## Implicit regularization
    ∂²L_∂x² = ∇²J(xᵢ) #+ ForwardDiff.jacobian(x -> ∂f(x)'λᵢ, xᵢ)
    ∂f_∂x = ∂f(xᵢ)

    ## KKT system
    H = [∂²L_∂x² ∂f_∂x'; ∂f_∂x 0]
    g = [∇J(xᵢ) + ∂f_∂x'λᵢ; f(xᵢ)]
    
    Δz = -H\g
    Δx = Δz[1:2]
    Δλ = Δz[3]
    return xᵢ .+ Δx, λᵢ .+ Δλ
end

In [ ]:
fig = Figure()
ax = Axis(fig[1,1], aspect=1)

## Initial guess
# xᵢ = Float64[-0.75; -1.75]
xᵢ = Float64[-3; 2]
λᵢ = Float64[0.0]

draw_contour(ax)
plot!(ax, [xᵢ[1]], [xᵢ[2]], color=:green, marker=:circle, markersize=15)

In [ ]:
xᵢ₊₁, λᵢ₊₁ = gauss_newton_step(xᵢ, λᵢ)
plot!(ax, [xᵢ₊₁[1]], [xᵢ₊₁[2]], color=:green, marker=:x, markersize=15)
xᵢ .= xᵢ₊₁
λᵢ .= λᵢ₊₁

fig

# Example II.2
-----
**How to interpret Piccolo outputs**

In [ ]:
system = QuantumSystem(0.01 * PAULIS[:Z], [PAULIS[:X], PAULIS[:Y]])
U_goal = GATES[:X]

T = 50
Δt = 0.2

In [ ]:
Random.seed!(1234)
ipopt_options = IpoptOptions(max_iter=50, recalc_y="yes", recalc_y_feas_tol=5e1)
problem = UnitarySmoothPulseProblem(system, U_goal, T, Δt, ipopt_options=ipopt_options)

Look for *inf_pr*, *inf_du*, and *complementarity*! 


In [ ]:
solve!(problem, max_iter=50)

From the IPOPT documentation (https://coin-or.github.io/Ipopt/OUTPUT.html):

- **inf_pr**: The unscaled constraint violation at the current point. This quantity is the infinity-norm (max) of the (unscaled) constraints ( gL≤g(x)≤gU in (NLP)). During the restoration phase, this value remains the constraint violation of the original problem at the current point. The option inf_pr_output can be used to switch to the printing of a different quantity.

- **inf_du**: The scaled dual infeasibility at the current point. This quantity measure the infinity-norm (max) of the internal dual infeasibility, Eq. (4a) in the implementation paper [12], including inequality constraints reformulated using slack variables and problem scaling. During the restoration phase, this is the value of the dual infeasibility for the restoration phase problem.

- **lg(mu)**: log10 of the value of the barrier parameter μ.

- **||d||**: The infinity norm (max) of the primal step (for the original variables x and the internal slack variables s). During the restoration phase, this value includes the values of additional variables, p and n (see Eq. (30) in [12]).

- **lg(rg)**: log10 of the value of the regularization term for the Hessian of the Lagrangian in the augmented system ( δw in Eq. (26) and Section 3.1 in [12]). A dash ("-") indicates that no regularization was done.

- **alpha_du**: The stepsize for the dual variables ( αzk in Eq. (14c) in [12]).

In [ ]:
unitary_fidelity(problem)

In [ ]:
plot_unitary_populations(problem)

# Exercises
-----

## Exercise II.1
**The Gauss-Newton Approximation**

This quick calculation should hopefully remind you about the Gauss-Newton approximation.

Start with a cost $J(\mathbf{x})$. The necessary condition for optimality is $\nabla J(\mathbf{x}) = 0$. Our journey starts by asking what happens if $J(\mathbf{x})$ is actually a least squares problem. For example, $J(\mathbf{x}) := \frac{1}{2}||\mathbf{g}(\mathbf{x})||_2^2$. 

**Exercise**
1. Compute the Newton step for $J$ in terms of $\mathbf{g}$.
2. There should be two terms in the Hessian. Drop the term that includes a derivative larger than a gradient. This is the Gauss-Newton approximation. When is it ok to drop this term?
3. Compare this to the `H` we computed in the KKT system, and see how our Gauss-Newton approximation of the system matches what we did here.

## Exercise II.2

**The augmented Lagrangian**

Augmented Lagrangians offload penalties onto Lagrange multipliers.

In [ ]:
function augmented_lagrangian(x, λ, ρ=1.0)
    p = max(0, f(x))
    return J(x) + λ*p + ρ/2 * p'p
end

This exercise is **UNDER CONSTRUCTION** -- check back soon!